In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

df = pd.read_csv('data/ebd_US-IL_200801_201212_relJan-2024.txt',sep='\t')
df = df.loc[:, ['CATEGORY', 'COMMON NAME', 'SCIENTIFIC NAME', 'OBSERVATION COUNT', 'EXOTIC CODE', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE', 'PROTOCOL TYPE', 'ALL SPECIES REPORTED']]


C:\Users\School\AppData\Local\Temp\ipykernel_17640\972917801.py:5: DtypeWarning: Columns (11,12,13,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/ebd_US-IL_200801_201212_relJan-2024.txt',sep='\t')


In [2]:
df.head()

,CATEGORY,COMMON NAME,SCIENTIFIC NAME,OBSERVATION COUNT,EXOTIC CODE,LATITUDE,LONGITUDE,OBSERVATION DATE,PROTOCOL TYPE,ALL SPECIES REPORTED
0,species,American White Pelican,Pelecanus erythrorhynchos,49,NaN,37.104636,-89.370953,2008-01-12,Historical,0
1,species,American Crow,Corvus brachyrhynchos,X,NaN,38.850907,-89.256706,2008-01-01,Traveling,1
2,species,American Goldfinch,Spinus tristis,X,NaN,38.850907,-89.256706,2008-01-01,Traveling,1
3,species,American Kestrel,Falco sparverius,X,NaN,38.850907,-89.256706,2008-01-01,Traveling,1
4,species,Bald Eagle,Haliaeetus leucocephalus,X,NaN,38.850907,-89.256706,2008-01-01,Traveling,1


## Filtering eBird Dataset

In [3]:
df['CATEGORY'].unique()

array(['species', 'issf', 'domestic', 'slash', 'form', 'spuh', 'hybrid',
       'intergrade'], dtype=object)

In [4]:
df['PROTOCOL TYPE'].unique()

array(['Historical', 'Traveling', 'Incidental', 'Stationary', 'Area',
       'International Shorebird Survey (ISS)', 'Banding',
       'Rusty Blackbird Spring Migration Blitz',
       'Standardized Yard Count', 'Greater Gulf Refuge Waterbird Count',
       'Nocturnal Flight Call Count'], dtype=object)

We will keep only species level observations (removing subspecies and genus level observations). We will also filter out incomplete checklists and incidental observations to manage bias towards specific species. 

In [5]:
# Applying filters
df = df[(df['CATEGORY']=='species') & (df['PROTOCOL TYPE']=='Traveling') | (df['PROTOCOL TYPE']=='Stationary') & (df['ALL SPECIES REPORTED']==1)]

In [6]:
df.head()

,CATEGORY,COMMON NAME,SCIENTIFIC NAME,OBSERVATION COUNT,EXOTIC CODE,LATITUDE,LONGITUDE,OBSERVATION DATE,PROTOCOL TYPE,ALL SPECIES REPORTED
1,species,American Crow,Corvus brachyrhynchos,X,NaN,38.850907,-89.256706,2008-01-01,Traveling,1
2,species,American Goldfinch,Spinus tristis,X,NaN,38.850907,-89.256706,2008-01-01,Traveling,1
3,species,American Kestrel,Falco sparverius,X,NaN,38.850907,-89.256706,2008-01-01,Traveling,1
4,species,Bald Eagle,Haliaeetus leucocephalus,X,NaN,38.850907,-89.256706,2008-01-01,Traveling,1
5,species,Blue Jay,Cyanocitta cristata,X,NaN,38.850907,-89.256706,2008-01-01,Traveling,1


## ebird Dataset Transformation

In [7]:
# Native column: 1 = is native to chicago, 0 = not native to chicago
def exotic_trans(row):
    if row == np.nan:
        return 0
    else:
        return 1

df['NATIVE'] = df['EXOTIC CODE'].apply(exotic_trans)

In [8]:
# Assume all 'X' observations have a count of 1 bird
def obs_trans(row):
    if row == 'X':
        return 1
    else:
        return row

df['COUNT'] = df['OBSERVATION COUNT'].apply(obs_trans)

In [9]:
# remove unnecessary columns
df = df.loc[:, ['COMMON NAME', 'SCIENTIFIC NAME', 'NATIVE', 'COUNT', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE']]
df.head()

,COMMON NAME,SCIENTIFIC NAME,NATIVE,COUNT,LATITUDE,LONGITUDE,OBSERVATION DATE
1,American Crow,Corvus brachyrhynchos,1,1,38.850907,-89.256706,2008-01-01
2,American Goldfinch,Spinus tristis,1,1,38.850907,-89.256706,2008-01-01
3,American Kestrel,Falco sparverius,1,1,38.850907,-89.256706,2008-01-01
4,Bald Eagle,Haliaeetus leucocephalus,1,1,38.850907,-89.256706,2008-01-01
5,Blue Jay,Cyanocitta cristata,1,1,38.850907,-89.256706,2008-01-01


## Aggregate eBird data based on neighborhood

In [10]:
com_areas = gpd.read_file('data/neighborhoods/geo_export_f5325bf0-9c6d-49a5-a5d9-0e5bf24fa856.shp')
com_areas = com_areas.loc[:,['community','geometry']]

In [11]:
com_areas.head()

,community,geometry
0,DOUGLAS,"POLYGON ((-87.60914 41.84469, -87.60915 41.844..."
1,OAKLAND,"POLYGON ((-87.59215 41.81693, -87.59231 41.816..."
2,FULLER PARK,"POLYGON ((-87.62880 41.80189, -87.62879 41.801..."
3,GRAND BOULEVARD,"POLYGON ((-87.60671 41.81681, -87.60670 41.816..."
4,KENWOOD,"POLYGON ((-87.59215 41.81693, -87.59215 41.816..."


In [12]:
len(com_areas)

77

In [13]:
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(df['LONGITUDE'], df['LATITUDE'])]
geo_df = gpd.GeoDataFrame(df, geometry=geometry)

In [14]:
geo_df.head()

,COMMON NAME,SCIENTIFIC NAME,NATIVE,COUNT,LATITUDE,LONGITUDE,OBSERVATION DATE,geometry
1,American Crow,Corvus brachyrhynchos,1,1,38.850907,-89.256706,2008-01-01,POINT (-89.25671 38.85091)
2,American Goldfinch,Spinus tristis,1,1,38.850907,-89.256706,2008-01-01,POINT (-89.25671 38.85091)
3,American Kestrel,Falco sparverius,1,1,38.850907,-89.256706,2008-01-01,POINT (-89.25671 38.85091)
4,Bald Eagle,Haliaeetus leucocephalus,1,1,38.850907,-89.256706,2008-01-01,POINT (-89.25671 38.85091)
5,Blue Jay,Cyanocitta cristata,1,1,38.850907,-89.256706,2008-01-01,POINT (-89.25671 38.85091)


In [15]:
ebird_gdf = gpd.sjoin(geo_df, com_areas, how='left', op='within')

C:\Users\School\anaconda3\envs\cs418env\Lib\site-packages\IPython\core\interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\School\AppData\Local\Temp\ipykernel_17640\1119340037.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: GEOGCS["WGS84(DD)",DATUM["WGS84",SPHEROID["WGS84", ...

  ebird_gdf = gpd.sjoin(geo_df, com_areas, how='left', op='within')


In [16]:
ebird_gdf.head()

,COMMON NAME,SCIENTIFIC NAME,NATIVE,COUNT,LATITUDE,LONGITUDE,OBSERVATION DATE,geometry,index_right,community
1,American Crow,Corvus brachyrhynchos,1,1,38.850907,-89.256706,2008-01-01,POINT (-89.25671 38.85091),NaN,NaN
2,American Goldfinch,Spinus tristis,1,1,38.850907,-89.256706,2008-01-01,POINT (-89.25671 38.85091),NaN,NaN
3,American Kestrel,Falco sparverius,1,1,38.850907,-89.256706,2008-01-01,POINT (-89.25671 38.85091),NaN,NaN
4,Bald Eagle,Haliaeetus leucocephalus,1,1,38.850907,-89.256706,2008-01-01,POINT (-89.25671 38.85091),NaN,NaN
5,Blue Jay,Cyanocitta cristata,1,1,38.850907,-89.256706,2008-01-01,POINT (-89.25671 38.85091),NaN,NaN


In [17]:
# remove observations outside of Chicago
ebird_gdf = ebird_gdf[ebird_gdf['community'].notna()]

In [18]:
# also remove more unnecessary columns
ebird_gdf = ebird_gdf.drop(columns=["index_right","LATITUDE","LONGITUDE","OBSERVATION DATE", "COMMON NAME"])

In [19]:
ebird_gdf.dtypes

SCIENTIFIC NAME      object
NATIVE                int64
COUNT                object
geometry           geometry
community            object
dtype: object

In [20]:
ebird_gdf['COUNT'] = ebird_gdf['COUNT'].astype(int)

In [21]:
# aggregate 
def agg_comm(series):
    return list(series)

grouped = ebird_gdf.groupby('community').agg(agg_comm)
grouped['NATIVE'] = grouped['NATIVE'].apply(lambda x: sum(x))

In [23]:
grouped = grouped.drop(columns=['geometry'])

In [24]:
grouped.head()

,SCIENTIFIC NAME,NATIVE,COUNT
community,,,
ALBANY PARK,"[Spinus tristis, Megaceryle alcyon, Cyanocitta...",1281,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 2, ..."
ARMOUR SQUARE,"[Turdus migratorius, Branta canadensis, Aegoli...",35,"[1, 1, 1, 1, 6, 3, 1, 1, 5, 2, 5, 1, 1, 1, 1, ..."
AUSTIN,"[Branta canadensis, Branta canadensis, Dryobat...",16232,"[375, 380, 1, 19, 2, 1, 6, 7, 1, 1, 48, 6, 1, ..."
AVONDALE,"[Corvus brachyrhynchos, Branta canadensis, Buc...",97,"[1, 45, 1, 12, 1, 1, 7, 3, 1, 1, 2, 34, 1, 77,..."
BELMONT CRAGIN,"[Setophaga ruticilla, Setophaga ruticilla, Tur...",123,"[2, 2, 10, 10, 1, 1, 2, 2, 1, 1, 2, 2, 10, 10,..."


## Join eBird and census datasets

In [25]:
census_df = pd.read_csv("data/Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012_20240228.csv")

In [26]:
census_df.head()

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


In [27]:
census_df['COMMUNITY AREA NAME'] = census_df['COMMUNITY AREA NAME'].str.upper()

In [28]:
final_df = census_df.merge(grouped, left_on='COMMUNITY AREA NAME', right_on='community')
final_df.head()

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,SCIENTIFIC NAME,NATIVE,COUNT
0,1.0,ROGERS PARK,7.7,23.6,8.7,18.2,27.5,23939,39.0,"[Spinus tristis, Turdus migratorius, Turdus mi...",676,"[1, 46, 4, 9, 1, 3, 1, 9, 12, 1, 2, 1, 2, 1, 7..."
1,2.0,WEST RIDGE,7.8,17.2,8.8,20.8,38.5,23040,46.0,"[Sturnus vulgaris, Aquila chrysaetos, Passer d...",255,"[7, 1, 5, 4, 100, 2, 2, 2, 1, 1, 1, 25, 25, 25..."
2,3.0,UPTOWN,3.8,24.0,8.9,11.8,22.2,35787,20.0,"[Corvus brachyrhynchos, Corvus brachyrhynchos,...",75132,"[5, 7, 3, 3, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,4.0,LINCOLN SQUARE,3.4,10.9,8.2,13.4,25.5,37524,17.0,"[Spinus tristis, Acanthis flammea, Junco hyema...",3751,"[1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,5.0,NORTH CENTER,0.3,7.5,5.2,4.5,26.2,57123,6.0,"[Corvus brachyrhynchos, Spinus tristis, Setoph...",1432,"[1, 7, 1, 28, 22, 4, 2, 5, 2, 1, 3, 2, 6, 2, 1..."
